In [ ]:
import spacy
from spacy.training import Example
import random

# Create a blank English model
nlp = spacy.blank("en")

# Add text classifier component
textcat = nlp.add_pipe("textcat")

# Add escalation labels
textcat.add_label("P0")
textcat.add_label("P1")
textcat.add_label("P2")



1

In [2]:
# Simple training data based on your escalation policy
TRAINING_DATA = [
    # P0 - Outage or data-loss risk
    ("Database is completely down", {"cats": {"P0": 1.0, "P1": 0.0, "P2": 0.0}}),
    ("Total system outage", {"cats": {"P0": 1.0, "P1": 0.0, "P2": 0.0}}),
    ("Data corruption detected", {"cats": {"P0": 1.0, "P1": 0.0, "P2": 0.0}}),
    ("Service unavailable for all users", {"cats": {"P0": 1.0, "P1": 0.0, "P2": 0.0}}),
    ("Critical data loss reported", {"cats": {"P0": 1.0, "P1": 0.0, "P2": 0.0}}),
    ("Complete system failure", {"cats": {"P0": 1.0, "P1": 0.0, "P2": 0.0}}),
    
    # P1 - Multiple customers blocked
    ("API failing for multiple customers", {"cats": {"P0": 0.0, "P1": 1.0, "P2": 0.0}}),
    ("Payment system down affecting many users", {"cats": {"P0": 0.0, "P1": 1.0, "P2": 0.0}}),
    ("Login issues reported by several customers", {"cats": {"P0": 0.0, "P1": 1.0, "P2": 0.0}}),
    ("Multiple users cannot access dashboard", {"cats": {"P0": 0.0, "P1": 1.0, "P2": 0.0}}),
    ("Bulk email delivery failing", {"cats": {"P0": 0.0, "P1": 1.0, "P2": 0.0}}),
    ("Regional service disruption", {"cats": {"P0": 0.0, "P1": 1.0, "P2": 0.0}}),
    
    # P2 - Single customer degradation
    ("One user reporting slow performance", {"cats": {"P0": 0.0, "P1": 0.0, "P2": 1.0}}),
    ("Individual account sync issue", {"cats": {"P0": 0.0, "P1": 0.0, "P2": 1.0}}),
    ("Single customer login problem", {"cats": {"P0": 0.0, "P1": 0.0, "P2": 1.0}}),
    ("Minor UI glitch for one user", {"cats": {"P0": 0.0, "P1": 0.0, "P2": 1.0}}),
    ("Customer reports feature not working", {"cats": {"P0": 0.0, "P1": 0.0, "P2": 1.0}}),
    ("Individual data export failing", {"cats": {"P0": 0.0, "P1": 0.0, "P2": 1.0}}),
]



In [3]:
def train_model():
    """Train the escalation prediction model"""
    print("Training escalation prediction model...")
    
    # Initialize the model
    nlp.initialize()
    
    # Training loop
    for i in range(20):  # Simple training - 20 iterations
        losses = {}
        random.shuffle(TRAINING_DATA)
        
        for text, annotations in TRAINING_DATA:
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update([example], losses=losses)
        
        if i % 5 == 0:
            print(f"Iteration {i}, Losses: {losses}")
    
    print("Training completed!")



In [4]:
def predict_escalation(incident_text):
    """Predict escalation level for given incident text"""
    doc = nlp(incident_text)
    
    # Get prediction scores
    scores = doc.cats
    
    # Find the highest scoring category
    predicted_level = max(scores, key=scores.get)
    confidence = scores[predicted_level]
    
    # Map to escalation policy
    escalation_map = {
        "P0": "Outage or data-loss risk — page on-call within 5 minutes.",
        "P1": "Multiple customers blocked — escalate to Incident Manager within 30 minutes.",
        "P2": "Single-customer degradation — open Jira and respond within 4 business hours."
    }
    
    return {
        "level": predicted_level,
        "confidence": round(confidence, 3),
        "action": escalation_map[predicted_level],
        "all_scores": {k: round(v, 3) for k, v in scores.items()}
    }

def test_model():
    """Test the model with sample incidents"""
    test_cases = [
        "Database server crashed, all services down",
        "Multiple customers cannot process payments",
        "One customer reports dashboard loading slowly",
        "Complete data center outage",
        "Several users experiencing login timeouts"
    ]
    
    print("\n=== Testing Escalation Predictions ===")
    for incident in test_cases:
        result = predict_escalation(incident)
        print(f"\nIncident: '{incident}'")
        print(f"Predicted: {result['level']} (confidence: {result['confidence']})")
        print(f"Action: {result['action']}")
        print(f"All scores: {result['all_scores']}")

def save_model(model_path="escalation_model"):
    """Save the trained model"""
    nlp.to_disk(model_path)
    print(f"\nModel saved to: {model_path}")

def load_model(model_path="escalation_model"):
    """Load a saved model"""
    loaded_nlp = spacy.load(model_path)
    print(f"Model loaded from: {model_path}")
    return loaded_nlp



In [5]:
if __name__ == "__main__":
    # Train the model
    train_model()
    
    # Test predictions
    test_model()
    
    # Save the model
    save_model()
    
    print("\n=== Usage Example ===")
    print("To use the model:")
    print("result = predict_escalation('Your incident description here')")
    print("print(result)")

Training escalation prediction model...
Iteration 0, Losses: {'textcat': 4.05171762406826}
Iteration 5, Losses: {'textcat': 4.104989801589909e-05}
Iteration 10, Losses: {'textcat': 3.938824139737562e-06}
Iteration 15, Losses: {'textcat': 1.234700728325322e-06}
Training completed!

=== Testing Escalation Predictions ===

Incident: 'Database server crashed, all services down'
Predicted: P0 (confidence: 0.994)
Action: Outage or data-loss risk — page on-call within 5 minutes.
All scores: {'P0': 0.994, 'P1': 0.001, 'P2': 0.006}

Incident: 'Multiple customers cannot process payments'
Predicted: P1 (confidence: 0.996)
Action: Multiple customers blocked — escalate to Incident Manager within 30 minutes.
All scores: {'P0': 0.001, 'P1': 0.996, 'P2': 0.004}

Incident: 'One customer reports dashboard loading slowly'
Predicted: P2 (confidence: 0.997)
Action: Single-customer degradation — open Jira and respond within 4 business hours.
All scores: {'P0': 0.0, 'P1': 0.003, 'P2': 0.997}

Incident: 'Comp

In [21]:
escalate = predict_escalation("Customer asks about nonprofit discount and annual prepay together.")

/Users/yeshwanthakula/Desktop/assignment/venv/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(
